## Data Cleansing and Feature Selection

In [19]:
import random
import pandas as pd
import numpy as np
import datetime

%matplotlib inline

In [86]:
path = '/home/altieris/docker/jupyter/notebooks/ceabs/accidents/data/input/raw/'

In [87]:
df = pd.read_csv(path +'accidents_brasil.csv', sep=';')
df.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,velocidade_via
0,99973.0,2018-01-01,segunda-feira,00:20:00,RJ,116.0,"303,5",RESENDE ...,Condutor Dormindo ...,Saída de leito carroçável ...,...,1,0,4,1,-22.469370,-44.447050,SR-RJ,DEL5/7,UOP03/RJ,50
1,99976.0,2018-01-01,segunda-feira,00:40:00,SC,282.0,"0,4",FLORIANOPOLIS ...,Não guardar distância de segurança ...,Colisão traseira ...,...,1,0,2,2,-27.599717,-48.575657,SR-SC,DEL8/1,UOP01/SC,100
2,99977.0,2018-01-01,segunda-feira,00:30:00,RJ,493.0,1,ITABORAI ...,Ultrapassagem Indevida ...,Colisão frontal ...,...,3,1,1,3,-22.763901,-42.927532,SR-RJ,DEL5/4,UOP02/RJ,50
3,99981.0,2018-01-01,segunda-feira,01:15:00,RS,386.0,134,SARANDI ...,Ingestão de Álcool ...,Colisão transversal ...,...,2,0,0,2,-27.953636,-52.916374,SR-RS,DEL9/14,UOP01/RS,60
4,99982.0,2018-01-01,segunda-feira,00:20:00,RS,293.0,"151,7",CANDIOTA ...,Falta de Atenção à Condução ...,Saída de leito carroçável ...,...,0,0,1,1,-31.395214,-53.783912,SR-RS,DEL9/11,UOP03/RS,80


In [88]:
df.index = pd.to_datetime(df.data_inversa)
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day

In [89]:
def regiao(uf):
    if uf in ('SP','RJ','MG','ES'):
        return 'SUDESTE'
    elif uf in ('AM','RR','AP','PA','TO','RO','AC'):
        return 'NORTE'
    elif uf in ('MA','PI','CE','RN','PE','PB','SE','AL','BA'):
        return 'NORDESTE'
    elif uf in ('MT','MS','GO','DF'):
        return 'CENTRO-OESTE'
    else:
        return 'SUL'

df['regiao'] = df.apply(lambda row: regiao(row['uf']), axis=1)

### Feriados 2017

| Data       	| Dia da semana    	|  Feriado                                  	|
|------------	|----------------	|--------------------------------------------	|
| 2017-1-1   	| domingo        	|  Confraternização Universal                	|
| 2017-2-27  	| segunda-feira  	| Carnaval                                   	|
| 2017-2-28  	| terça-feira    	| Carnaval                                   	|
| 2017-4-14  	| sexta-feira    	| Paixão de Cristo                           	|
| 2017-4-21  	| sexta-feira    	| Tiradentes                                 	|
| 2017-5-1   	| segunda-feira  	| Dia do Trabalho                            	|
| 2017-6-15  	| quinta-feira   	| Corpus Christi                             	|
| 2017-9-7   	| quinta-feira   	| Independência do Brasil                    	|
| 2017-10-12 	| quinta-feira   	| Nossa Sr.a Aparecida - Padroeira do Brasil 	|
| 2017-11-2  	| quinta-feira   	| Finados                                    	|
| 2017-11-15 	| quarta-feira   	| Proclamação da República                   	|
| 2017-12-25 	| segunda-feira  	| Natal                                      	|

### Feriados 2018

| Data       	| Dia da Semana 	| Feriado                                    	|
|------------	|---------------	|--------------------------------------------	|
| 2018-1-1   	| segunda-feira 	| Confraternização Universal                 	|
| 2018-2-12  	| segunda-feira 	| Carnaval                                   	|
| 2018-2-13  	| terça-feira   	| Carnaval                                   	|
| 2018-3-30  	| sexta-feira   	| Paixão de Cristo                           	|
| 2018-4-21  	| sábado        	| Tiradentes                                 	|
| 2018-5-1   	| terça-feira   	| Dia do Trabalho                            	|
| 2018-5-31  	| quinta-feira  	| Corpus Christi                             	|
| 2018-9-7   	| sexta-feira   	| Independência do Brasil                    	|
| 2018-10-12 	| sexta-feira   	| Nossa Sr.a Aparecida - Padroeira do Brasil 	|
| 2018-11-2  	| sexta-feira   	| Finados                                    	|
| 2018-11-15 	| quinta-feira  	| Proclamação da República                   	|
| 2018-12-25 	| terça-feira   	| Natal                                      	|

In [90]:
def feriado(data):
    if data in ('2017-01-01','2017-02-27','2017-02-28','2017-04-14','2017-04-21','2017-05-01','2017-06-15','2017-09-07','2017-10-12','2017-11-02','2017-11-15','2017-12-25'): # feriados 2017
        return 1
    elif data in ('2018-01-01','2018-02-12','2018-02-13','2018-03-30','2018-04-21','2018-05-01','2018-05-31','2018-09-07','2018-10-12','2018-11-02','2018-11-15','2018-12-25'): # feriados 2018
        return 1
    else:
        return 0


def desc_feriado(data):
    if data in ('2017-01-01','2018-01-01'):
        return 'Confraternização Universal'
    elif data in ('2017-02-27','2017-02-28','2018-02-12','2018-02-13'):
        return 'Carnaval'
    elif data in ('2017-04-14','2018-03-30'):
        return 'Paixão de Cristo'
    elif data in ('2017-04-21','2018-04-21'):
        return 'Tiradentes'
    elif data in ('2017-05-1','2018-05-1'):
        return 'Dia do Trabalho'
    elif data in ('2017-06-15','2018-05-31'):
        return 'Corpus Christi'
    elif data in ('2017-09-7','2018-09-7'):
        return 'Independência do Brasil'
    elif data in ('2017-10-12','2018-10-12'):
        return 'Nossa Sr.a Aparecida - Padroeira do Brasil'
    elif data in ('2017-11-02','2018-11-02'):
        return 'Finados'
    elif data in ('2017-11-15','2018-11-15'):
        return 'Proclamação da República'
    elif data in ('2017-12-25','2018-12-25'):
        return 'Natal'
    else:
        return ''
    
df['feriado'] = df.apply(lambda row: feriado(row['data_inversa']), axis=1)
df['nome_feriado'] = df.apply(lambda row: desc_feriado(row['data_inversa']), axis=1)

http://www.iag.usp.br/astronomia/inicio-das-estacoes-do-ano

### Estações do Ano 2017
Outono     2017-03-20  
Inverno    2017-06-21  
Primavera  2017-09-22  
Verão      2017-12-21  

### Estações do Ano 2018
Outono     2018-03-20  
Inverno    2018-06-21  
Primavera  2018-09-22  
Verão      2018-12-21  


In [91]:
def desc_estacao_ano(data):
    
    if (data >= '2016-12-21' and  data < '2017-03-20'):
        return 'verão'
    elif (data >= '2017-03-20' and data < '2017-06-21') or (data >= '2018-03-20' and data < '2018-06-21'):
        return 'outono'
    elif (data >= '2017-06-21' and data < '2017-09-22') or (data >= '2018-06-21' and data < '2018-09-22'):
        return 'inverno'
    elif (data >= '2017-09-22' and data < '2017-12-21')  or (data >= '2018-09-22' and data < '2018-12-21'):
        return 'primavera'
    elif (data >= '2017-12-21'  and data < '2018-03-20') or (data >= '2018-12-21' and data < '2019-03-20'):
        return 'verão'
    else:
        return ''          
                 
df['estacao'] = df.apply(lambda row: desc_estacao_ano(row['data_inversa']), axis=1)

In [92]:
df.tail()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,delegacia,uop,velocidade_via,year,month,day,regiao,feriado,nome_feriado,estacao
data_inversa,,,,,,,,,,,,,,,,,,,,,
2017-09-10,133285.0,2017-09-10,domingo,11:50:00,AL,316.0,242,ATALAIA ...,Falta de Atenção à Condução ...,Queda de ocupante de veículo ...,...,DEL13/1,UOP01/AL,60,2017,9,10,NORDESTE,0,,inverno
2017-10-18,133485.0,2017-10-18,quarta-feira,17:20:00,PR,277.0,5,PARANAGUA ...,Falta de Atenção do Pedestre ...,Atropelamento de Pedestre ...,...,DEL7/1,UOP07/PR,60,2017,10,18,SUL,0,,primavera
2017-12-10,133723.0,2017-12-10,domingo,10:05:00,MT,364.0,"233,5",RONDONOPOLIS ...,Não guardar distância de segurança ...,Saída de leito carroçável ...,...,DEL2/2,UOP01/MT,110,2017,12,10,CENTRO-OESTE,0,,primavera
2017-02-10,133740.0,2017-02-10,sexta-feira,15:50:00,MG,50.0,61,ARAGUARI ...,Não guardar distância de segurança ...,Colisão traseira ...,...,DEL4/17,UOP02/MG,60,2017,2,10,SUDESTE,0,,verão
2017-12-20,133958.0,2017-12-20,quarta-feira,05:00:00,PE,408.0,84,SAO LOURENCO DA MATA ...,Falta de Atenção à Condução ...,Capotamento ...,...,DEL11/1,UOP01/PE,110,2017,12,20,NORDESTE,0,,primavera


In [93]:
# norte 5.271944, -60.2125
# sul -84, 79.494167
# leste -20.461944, -28.835833
# oeste -7.535833, -73.990556

df = df[(df['latitude'] > -90) & (df['latitude'] < 90)]
df = df[(df['longitude'] > -180) & (df['longitude'] < 180)]

In [94]:
sul = df[(df['regiao']  == 'SUL')]
sudeste = df[(df['regiao']  == 'SUDESTE')]
centro_oeste = df[(df['regiao']  == 'CENTRO-OESTE')]
norte = df[(df['regiao']  == 'NORTE')]
nordeste = df[(df['regiao']  == 'NORDESTE')]

In [95]:
sul = sul[(sul['latitude'] != 0)]
sul = sul[(sul['latitude'] != -2.70895468)]
sul = sul[(sul['latitude'] != -11.59843162)]
sul = sul[(sul['latitude'] != 26)]
sul = sul[(sul['latitude'] != -22.00960734)] 
sul = sul[(sul['latitude'] != -26)]  

In [96]:
sudeste = sudeste[(sudeste['latitude'] != -19)]
sudeste = sudeste[(sudeste['latitude'] != 19)]
sudeste = sudeste[(sudeste['latitude'] != 23)]
sudeste = sudeste[(sudeste['latitude'] != 0)]
sudeste = sudeste[(sudeste['latitude'] != -17.39257927)]

In [97]:
centro_oeste = centro_oeste[(centro_oeste['latitude'] != -9.10209674)] 
centro_oeste = centro_oeste[(centro_oeste['latitude'] != -5.0446853)] 
centro_oeste = centro_oeste[(centro_oeste['latitude'] != -5.05059064)]  
centro_oeste = centro_oeste[(centro_oeste['longitude'] != -42.7662325)]
centro_oeste = centro_oeste[(centro_oeste['longitude'] != -42.7662325)]

In [98]:
nordeste = nordeste[(nordeste['longitude'] != -94.49999571)] 
nordeste = nordeste[(nordeste['latitude'] < 0)] 
nordeste = nordeste[(nordeste['latitude'] != -19.31114336)] 
nordeste = nordeste[(nordeste['latitude'] != -85.0494645)] 

In [99]:
path = '/home/altieris/docker/jupyter/notebooks/ceabs/accidents/data/regioes/'

In [100]:
sul.to_csv(path+'accidents_sul.csv', sep=';', encoding='utf-8',index=False)

In [101]:
sudeste.to_csv(path+'accidents_sudeste.csv', sep=';', encoding='utf-8',index=False)


In [102]:
centro_oeste.to_csv(path+'accidents_centro_oeste.csv', sep=';', encoding='utf-8',index=False)


In [103]:
norte.to_csv(path+'accidents_norte.csv', sep=';', encoding='utf-8',index=False)


In [104]:
nordeste.to_csv(path+'accidents_nordeste.csv', sep=';', encoding='utf-8',index=False)

In [105]:
df1 = sul.append(sudeste)
df2 = df1.append(centro_oeste)
df3 = df2.append(norte)
df4 = df3.append(nordeste)
df4.shape

(119533, 38)

In [106]:
df4.to_csv(path+'accidents_brasil_cleansed.csv', sep=';', encoding='utf-8',index=False)